In [2]:
!pip install transformers
!pip install keras
!pip install tensorflow
!pip install jupyter-resource-usage
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master


     |████████████████████████████████| 42 kB 241 kB/s eta 0:00:011
     |████████████████████████████████| 377 kB 3.7 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 3.2 MB/s  eta 0:00:01
     |████████████████████████████████| 46.9 MB 55.6 MB/s eta 0:00:01
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-ql_rcvfv
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-ql_rcvfv
  Created wheel for kobert: filename=kobert-0.1.2-py3-none-any.whl size=12704 sha256=8bb142905a5bf3c4ea5bd78dc5040ec04f9b7474f4753a6e9da9b557602f1e7e
  Stored in directory: /tmp/pip-ephem-wheel-cache-0l1v49ay/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert


In [3]:
!wget https://raw.githubusercontent.com/yunakim2/OpenSourceProject/develop/data/labeled/samsung_2010_2021_01.csv

import gc
import torch

from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
import gluonnlp as nlp
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

import numpy as np
import random
import time
import datetime

import io
import pandas as pd
import torch
import torch.nn as nn
import torchvision.datasets as dsets

--2021-05-04 14:54:02--  https://raw.githubusercontent.com/yunakim2/OpenSourceProject/develop/data/labeled/samsung_2010_2021_01.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 41356492 (39M) [text/plain]
Saving to: ‘samsung_2010_2021_01.csv’

samsung_2010_2021_0 100%[===================>]  39.44M  77.9MB/s    in 0.5s    

2021-05-04 14:54:03 (77.9 MB/s) - ‘samsung_2010_2021_01.csv’ saved [41356492/41356492]



In [15]:
USE_CUDA = True
RANDOM_SEED=43 # 재현을 위해 랜덤시드 고정
TOKEN_MAX_LEN = 128*4
BATCH_SIZE = 8 #로컬 6GB일때 1, 클라우드T4 16GB일때 12
STATUS_PRINT_INTERVAL=25

random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed_all(RANDOM_SEED)

# 정확도 계산 함수
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


def format_time(elapsed):
    elapsed_rounded = int(round(elapsed))
    return str(datetime.timedelta(seconds=elapsed_rounded))

if USE_CUDA and torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

bertmodel, vocab = get_pytorch_kobert_model()

#%%


There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB
using cached model
using cached model


In [16]:
data = pd.read_csv('samsung_2010_2021_01.csv',encoding = 'utf-8')
test_cnt = int(data.shape[0] * 0.25)

test = data[:test_cnt]
train = data[test_cnt:]

tok = nlp.data.BERTSPTokenizer(get_tokenizer(), vocab, lower=False)
tokenizer = nlp.data.BERTSentenceTransform(tok,max_seq_length=TOKEN_MAX_LEN,vocab=vocab,pair=False)

print('train&validation data processing')
input_ids = np.array([tokenizer([i])[0] for i in train['text']]).astype(int)
print(vocab.to_tokens([int(i) for i in input_ids[0][:20]]))

''' 어텐션 마스크 '''
attention_mask = []
for seq in input_ids:
    seq_mask = [float(i > 0) for i in seq]
    attention_mask.append(seq_mask)

# print(attention_mask[0])

'''train&test validation set 분리'''
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, train['label'].values, random_state=RANDOM_SEED, test_size=0.1)
train_masks, validation_masks, _, _ = train_test_split(attention_mask,input_ids,random_state=RANDOM_SEED,test_size=0.1)

train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=BATCH_SIZE)

using cached model
train&validation data processing
['[CLS]', '▁삼성', '▁', '로', '고', '▁자료', '사진', '▁W', 'S', 'J', '칼', '럼', '▁경쟁', '▁치열', '해', '▁장', '담', '▁못해', '▁모멘텀', '▁']


In [17]:
print('test data processing')
input_ids = np.array([tokenizer([i])[0] for i in test['text']]).astype(int)
print(vocab.to_tokens([int(i) for i in input_ids[0][:20]]))


attention_masks = []
for seq in input_ids:
    seq_mask = [float(i > 0) for i in seq]
    attention_masks.append(seq_mask)

test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(test['label'].values)
test_masks = torch.tensor(attention_masks)

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=BATCH_SIZE)

test data processing
['[CLS]', '▁올해', '▁100', '년', '▁기업', '을', '▁향한', '▁비', '전', '20', '20', '▁', '을', '▁구', '체', '화', '하기', '▁위해', '▁초', '경쟁력']


In [18]:
"""분류를 위한 BERT 모델 생성"""

print('분류를 위한 BERT 모델 생성 작업 중 - ')
model = BertForSequenceClassification.from_pretrained(bertmodel.config.name_or_path, num_labels=2)
model.double()

"""학습 스케줄링"""
print('학습 스케줄링 중 - ')
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
epochs = 4
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

분류를 위한 BERT 모델 생성 작업 중 - 


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /root/kobert/kobert_from_pretrained and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


학습 스케줄링 중 - 


In [ ]:
# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()
device = "cuda:0"
model = model.to(device)

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    gc.collect()
    torch.cuda.empty_cache()
    

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
        

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
          # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)

        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)  
    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

     # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...
  Batch   500  of  1,042.    Elapsed: 0:06:32.
  Batch 1,000  of  1,042.    Elapsed: 0:13:03.

  Average training loss: 0.69
  Training epcoh took: 0:13:35

Running Validation...
  Accuracy: 0.55
  Validation took: 0:00:28

======== Epoch 2 / 4 ========
Training...
  Batch   500  of  1,042.    Elapsed: 0:06:32.


In [13]:
# 시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    print(batch)
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch

    # 그래디언트 계산 안함
    with torch.no_grad():
        # Forward 수행
        outputs = model(b_input_ids,token_type_ids=None, attention_mask=b_input_mask)

    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy / nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))

RuntimeError: Input, output and indices must be on the current device